In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.7 MB/s eta 0:00:00


In [ ]:
# Import the Transformers pipeline library

from transformers import pipeline

# Preprocessing and visualization libraries
import plotly.express as px
import pandas as pd
import numpy as np
import textwrap

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install the required libraries

In [ ]:
!pip install plotly==5.14.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 65.8 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.13.1
    Uninstalling plotly-5.13.1:
      Successfully uninstalled plotly-5.13.1


In [ ]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Install Tensorflow.
Follow instructions here:
https://www.tensorflow.org/install
I did !pip install tensorflow on my miniconda prompt

In [ ]:
wrapper = textwrap.TextWrapper(width=80)

# This is a very good zero-shot classifier trained on 100 languages.
Check the model further in:https://huggingface.co/joeddav/xlm-roberta-large-xnli

## Download the model

In [ ]:
#zsmlc_classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli')

model_name = "facebook/bart-large-mnli"
zsmlc_classifier = pipeline("zero-shot-classification", model=model_name)

# Exampole of how the model works with different languages



In [ ]:
# we will classify the Russian translation of, "Who are you voting for in 2020?"
sequence_to_classify = "Do you know who you will vote for in the coming elections?"
# we can specify candidate labels in Russian or any other language above:
candidate_labels = ["Europe", "public health", "politics"]
results = zsmlc_classifier(sequence_to_classify, candidate_labels)
print(results)

{'sequence': 'Do you know who you will vote for in the coming elections?', 'labels': ['politics', 'Europe', 'public health'], 'scores': [0.9676707983016968, 0.021573791280388832, 0.010755473747849464]}


In [ ]:
# we will classify the Russian translation of, "Who are you voting for in 2020?"
sequence_to_classify = "За кого вы голосуете в 2020 году?"
# we can specify candidate labels in Russian or any other language above:
candidate_labels = ["Europe", "public health", "politics"]
zsmlc_classifier(sequence_to_classify, candidate_labels)

{'sequence': 'За кого вы голосуете в 2020 году?',
 'labels': ['politics', 'Europe', 'public health'],
 'scores': [0.4130873680114746, 0.3789117932319641, 0.20800085365772247]}

# Create some functions

In [ ]:
# function to choose the label with the highest probability

def make_prediction(clf_result):

    # Get the index of the maximum probability score
    max_index = np.argmax(clf_result["scores"])
    predicted_label = clf_result["labels"][max_index]

    return predicted_label

In [ ]:
# function to select a subset of data for this tutorial to make things faster,
# You don't have to use this. You can just slice the data

def select_subset_data(data, label_column, each_target_size = 2):

    all_targets = list(data[label_column].unique())
    list_dataframes = []

    for label in all_targets:
        subset = data[data[label_column]==str(label)]
        subset = subset.sample(each_target_size)

        list_dataframes.append(subset)

    return pd.concat(list_dataframes)

In [ ]:
# a function to predict the sequence with the zero-shot classifier and print the probabilites of each label

def run_batch_prediction(original_data, label_column, desc_col, my_classifier = zsmlc_classifier):

    # Make a copy of the data
    data_copy = original_data.copy()

    # The list that will contain the models predictions
    final_list_labels = []
    probabilities = []

    for index in range(len(original_data)):
    # Run classification
        sequences = original_data.iloc[index][desc_col]
        candidate_labels = list(original_data[label_column].unique())
        result = my_classifier(sequences, candidate_labels, multi_class = True)

    # Make prediction
        final_list_labels.append(make_prediction(result))
        probabilities.append(max(result['scores']))

    # Create the new column for the predictions
    data_copy["clf_predictions"] = final_list_labels
    data_copy["probabilities"] = probabilities

    return data_copy

# Try with a sample of the REDDIT English dataset

In [ ]:
!pwd

/content


In [ ]:
# this is a subset of the original data
# Replace 'filename.csv' with the actual name of your CSV file
#new_df = pd.read_csv('training_datasmall.csv', error_bad_lines=False)
#new_df.head()


# Read CSV file from Google Drive
new_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SurreyUniversity/training_datasmall.csv', error_bad_lines=False)
new_df.head()


<ipython-input-15-b0063e8367c5>:8: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  new_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SurreyUniversity/training_datasmall.csv', error_bad_lines=False)


,Unnamed: 0,subreddit,post,categories
0,0,bipolarreddit,This seroquel withdrawal sucks Problems sleepi...,bipolar
1,1,bipolarreddit,Mental/emotional pain What do you do when you ...,bipolar
2,2,bipolarreddit,Bipolar and different perspectives? Hi everyon...,bipolar
3,3,bipolarreddit,2019 resolution: be S.T.A.B.L.E. Hey friends! ...,bipolar
4,4,bipolarreddit,"Enjoy life with this one weird trick. Yes, you...",bipolar


THIS PART IS MACHINE TRANSLATION - NO NEED TO RUN

In [ ]:
from sklearn.utils import shuffle

In [ ]:
new_df = new_df.sample(frac=1, random_state=42)

In [ ]:
#Group the dataset by category
grouped = new_df.groupby('categories')


In [ ]:
small_new_df = pd.DataFrame(columns=new_df.columns)

In [ ]:
n_samples = 10000 // len(grouped)  # Number of samples per category
for _, group in grouped:
    small_new_df = small_new_df.append(group.sample(n=n_samples, random_state=42))


<ipython-input-27-7ac24384584b>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  small_new_df = small_new_df.append(group.sample(n=n_samples, random_state=42))
<ipython-input-27-7ac24384584b>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  small_new_df = small_new_df.append(group.sample(n=n_samples, random_state=42))
<ipython-input-27-7ac24384584b>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  small_new_df = small_new_df.append(group.sample(n=n_samples, random_state=42))
<ipython-input-27-7ac24384584b>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  small_new_df = small_new_df.append(group.sample(n=n_samples, random_state=42))


In [ ]:
small_new_df = small_new_df.sample(frac=1, random_state=42)

In [ ]:
small_new_df.reset_index(drop=True, inplace=True)

In [ ]:
print(small_new_df.head(50))

   Unnamed: 0      subreddit  \
0       17322     depression   
1      251841  bipolarreddit   
2      211827        anxiety   
3        3647  bipolarreddit   
4        2168  bipolarreddit   
5       19556     depression   
6      220994        anxiety   
7       24630     depression   
8       22567     depression   
9      222512        anxiety   
10       3701  bipolarreddit   
11      19608     depression   
12      20759     depression   
13      23112     depression   
14     247782  bipolarreddit   
15      18752     depression   
16     224447        anxiety   
17     455773      parenting   
18      20185     depression   
19     455364      parenting   
20     109110      parenting   
21     222285        anxiety   
22       5693  bipolarreddit   
23     107490      parenting   
24     249520  bipolarreddit   
25     455346      parenting   
26     213041        anxiety   
27     108083      parenting   
28       6032     depression   
29     108198      parenting   
30     2

In [ ]:
small_new_df.to_csv('small_dataset.csv', index=False)

In [ ]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 11.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16352 sha256=74dcc2c29ddcfb79c823378bd0aa6ae4d42edb3346dd0e1e9d7f83733261c6ec
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans

In [ ]:
from googletrans import Translator
translator = Translator()

In [ ]:
import googletrans

In [ ]:
!pip install googletrans==4.0.0-rc1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=4538035850e9f047f9a986751cba781491812b46670f44e2bcb7348bfa544531
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.1.0a0
    Uninstalling googletrans-3.1.0a0:
      Successfully uninstalled googletrans-3.1.0a0


In [ ]:
import pandas as pd
import random
from time import sleep
from googletrans import Translator

# Load the CSV dataset
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SurreyUniversity/small_dataset_10000.csv', encoding='UTF-8', error_bad_lines=False)

# Create a translator object
translator = Translator()

# Define the target language
target_lang = 'tr'  # Turkish language code

# Select the first 50 rows
rows_to_translate = dataset.head(50)

# Iterate through each row
for index, row in rows_to_translate.iterrows():
    # Get the post from the 'post' column
    post = row['post']

    # Translate the post to Turkish
    translation = translator.translate(post, dest=target_lang).text

    # Update the 'post' column with the translated text
    dataset.at[index, 'post'] = translation

    # Add a random time sleep between translations (e.g., 0.2 to 1 seconds)
    sleep(random.uniform(0.2, 1))

# Print the first 50 translated sentences
translated_sentences = dataset['post'].head(50)
print(translated_sentences)

# Save the translated dataset to a new CSV file
#dataset.to_csv('/content/drive/MyDrive/Colab Notebooks/SurreyUniversity/translated_file.csv', index=False)


<ipython-input-38-0a6ade724a25>:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SurreyUniversity/small_dataset_10000.csv', encoding='UTF-8', error_bad_lines=False)


0     Bunu benim gibi mücadele edenler için bırakmak...
1     EDMR tedavisi aylarca denedikten sonra nihayet...
2     Nörotransmitter Testi veya diğer testler Herha...
3     Biraz uykusuz kaldığında daha 'normal' hissede...
4     Anksiyete için Buspirone? Bunu endişe için ala...
5     Uyuşmuş... Depresyon bir kaltaktır. Yaklaşık 1...
6     Dün gece 5 yıl sonra ilk kez Seroquel'siz yatt...
7     Derin bir üzüntü halinde olan başka biri aktif...
8     "Hiç BENİM sorunumu düşündün mü?" (r/bipolar'd...
9     Bu herkese oluyor mu? Pekala, bu garip olabili...
10    Abilify "yürüme" sorusu Bunun sormak için iyi ...
11    Dönecek kimsem yokmuş gibi hissediyorum. Şimdi...
12    Eski sevgilim çocuğuma hamile olduğunu düşünme...
13    Pristiq para çekme işlemlerinden geçen veya ge...
14    5 yıllık yararsız MDB tedavisinden sonra bugün...
15    Danışmanım benden hoşlanıyor diye endişeleniyo...
16    Kaygıdan kaynaklanan fiziksel ağrılarla nasıl ...
17    Çocuğumun görünüşü nasıl olacak (Güney Kor

In [ ]:
# Iterate through each row
for index, row in dataset.iterrows():
    # Get the post and categories from the respective columns
    post = row['post']

    # Translate the post and categories to Turkish
    translation_post = translator.translate(post, dest=target_lang).text

    # Update the 'post' and 'categories' columns with the translated text
    dataset.at[index, 'post'] = translation_post

    # Add a random time sleep between translations (e.g., 0.2 to 1 seconds)
    sleep(random.uniform(0.2, 1))

# Print the first 50 translated sentences
translated_sentences = dataset[['post']].head(50)
print(translated_sentences)

# Save the translated dataset to a new CSV file
dataset.to_csv('/content/drive/MyDrive/Colab Notebooks/SurreyUniversity/translated_file.csv', index=False)


                                                 post        categories
0   Bunu benim gibi mücadele edenler için bırakmak...         depresyon
1   EDMR tedavisi aylarca denedikten sonra nihayet...       iki kutuplu
2   Nörotransmitter Testi veya diğer testler Herha...            endişe
3   Biraz uykusuz kaldığında daha 'normal' hissede...    çift ​​kutuplu
4   Anksiyete için Buspirone? Bunu endişe için ala...    çift ​​kutuplu
5   Uyuşmuş... Depresyon bir kaltaktır. Yaklaşık 1...         depresyon
6   Dün gece 5 yıl sonra ilk kez Seroquel'siz yatt...            endişe
7   Derin bir üzüntü halinde olan başka biri aktif...         depresyon
8   "Hiç BENİM sorunumu düşündün mü?" (r/bipolar'd...         depresyon
9   Bu herkese oluyor mu? Pekala, bu garip olabili...            endişe
10  Abilify "yürüme" sorusu Bunun sormak için iyi ...       iki kutuplu
11  Dönecek kimsem yokmuş gibi hissediyorum. Şimdi...         depresyon
12  Eski sevgilim çocuğuma hamile olduğunu düşünme...         de

In [ ]:
len(new_df)

66580

In [ ]:
# Get the subset of dataframe
subset_news_data = select_subset_data(new_df, "categories")

In [ ]:
len(subset_news_data)

8

In [ ]:
# Run the predictions on the new dataset
pred_res_data = run_batch_prediction(subset_news_data, "categories", "post")

pred_res_data

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transf

,Unnamed: 0,subreddit,post,categories,clf_predictions,probabilities
665,665,bipolarreddit,Weight Loss &amp; Bipolar My care provider and...,bipolar,bipolar,0.940764
1156,1156,bipolarreddit,How did you out your illness to your family? (...,bipolar,bipolar,0.989497
23596,23596,depression,I feel like I can’t breathe. I’m in the middle...,depression,depression,0.913695
17636,17636,depression,I Lost The Best Part of My Life 12/30/17. My f...,depression,depression,0.988572
64609,454620,parenting,Sex Ed book - The Amazing True Story of how ba...,non-mental,non-mental,0.537604
26193,107753,parenting,I am an awful person because I don’t believe i...,non-mental,bipolar,0.993183
29758,209350,anxiety,Dropping in a resume with anxiety I have prett...,anxiety,anxiety,0.996837
38735,218327,anxiety,What's your BIGGEST REGRET in life? I have ask...,anxiety,bipolar,0.671109


## Follow the following:
1. Compare the column 'categories' with the column 'clf_predictions'.
2. Find descripant labels
3. If the 'probabilities' is pretty high (choose a threshold), extract instances for manual analysis


# Example with Turkish

In [ ]:
# this is a subset of the original data

#arabic = pd.read_csv('./data/tweets_withpredictions.csv',index_col=0)
#arabic.rename(columns={"Tweet.1": "text",'predictions':'categories'}, inplace=True)
#arabic.head()

# Read CSV file from Google Drive
#turkish_df = pd.read_csv('/content/drive/MyDrive/SurreyUniversity/training_datasmall_2000.csv', error_bad_lines=False)
#turkish_df.head()

# Read CSV file from Google Drive
#turkish_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SurreyUniversity/training_datasmall_2000.csv', error_bad_lines=False)
#turkish_df.head()


# Read CSV file into DataFrame
turkish_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SurreyUniversity/translated_file_10000_tur.csv', encoding='UTF-8', error_bad_lines=False)

# Display the first few rows of the DataFrame
turkish_df.head()


<ipython-input-30-c0bfde52282a>:17: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  turkish_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SurreyUniversity/translated_file_10000_tur.csv', encoding='UTF-8', error_bad_lines=False)


,Unnamed: 0,subreddit,post,categories
0,17322,depression,Bunu benim gibi mücadele edenler için bırakmak...,depression
1,251841,bipolarreddit,EDMR tedavisi aylarca denedikten sonra nihayet...,bipolarreddit
2,211827,anxiety,Nörotransmitter Testi veya diğer testler Herha...,anxiety
3,3647,bipolarreddit,Biraz uykusuz kaldığında daha 'normal' hissede...,bipolarreddit
4,2168,bipolarreddit,Anksiyete için Buspirone? Bunu endişe için ala...,bipolarreddit


In [ ]:
print(turkish_df.columns)

Index(['Unnamed: 0', 'subreddit', 'post', 'categories'], dtype='object')


In [ ]:
import pandas as pd

# Assuming you have already read the CSV file into the `turkish_df` DataFrame
turkish = turkish_df[['categories','post']]

# Display the first few rows of the DataFrame
turkish.head()


,categories,post
0,depression,Bunu benim gibi mücadele edenler için bırakmak...
1,bipolarreddit,EDMR tedavisi aylarca denedikten sonra nihayet...
2,anxiety,Nörotransmitter Testi veya diğer testler Herha...
3,bipolarreddit,Biraz uykusuz kaldığında daha 'normal' hissede...
4,bipolarreddit,Anksiyete için Buspirone? Bunu endişe için ala...


In [ ]:
# Get the subset of dataframe for speed
subset_tr_data = turkish[:10]

In [ ]:
subset_tr_data

,categories,post
0,depression,Bunu benim gibi mücadele edenler için bırakmak...
1,bipolarreddit,EDMR tedavisi aylarca denedikten sonra nihayet...
2,anxiety,Nörotransmitter Testi veya diğer testler Herha...
3,bipolarreddit,Biraz uykusuz kaldığında daha 'normal' hissede...
4,bipolarreddit,Anksiyete için Buspirone? Bunu endişe için ala...
5,depression,Uyuşmuş... Depresyon bir kaltaktır. Yaklaşık 1...
6,anxiety,Dün gece 5 yıl sonra ilk kez Seroquel'siz yatt...
7,depression,Derin bir üzüntü halinde olan başka biri aktif...
8,depression,"""Hiç BENİM sorunumu düşündün mü?"" (r/bipolar'd..."
9,anxiety,"Bu herkese oluyor mu? Pekala, bu garip olabili..."


In [ ]:
# Run the predictions on the new dataset
pred_tr_data = run_batch_prediction(subset_tr_data, "categories","post")
pred_tr_data

The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transformers.
The `multi_class` argument has been deprecated and renamed to `multi_label`. `multi_class` will be removed in a future version of Transf

,categories,post,clf_predictions,probabilities
0,depression,Bunu benim gibi mücadele edenler için bırakmak...,bipolarreddit,0.724003
1,bipolarreddit,EDMR tedavisi aylarca denedikten sonra nihayet...,anxiety,0.114921
2,anxiety,Nörotransmitter Testi veya diğer testler Herha...,anxiety,0.145923
3,bipolarreddit,Biraz uykusuz kaldığında daha 'normal' hissede...,depression,0.249421
4,bipolarreddit,Anksiyete için Buspirone? Bunu endişe için ala...,anxiety,0.387436
5,depression,Uyuşmuş... Depresyon bir kaltaktır. Yaklaşık 1...,depression,0.979057
6,anxiety,Dün gece 5 yıl sonra ilk kez Seroquel'siz yatt...,bipolarreddit,0.501206
7,depression,Derin bir üzüntü halinde olan başka biri aktif...,anxiety,0.094733
8,depression,"""Hiç BENİM sorunumu düşündün mü?"" (r/bipolar'd...",bipolarreddit,0.984088
9,anxiety,"Bu herkese oluyor mu? Pekala, bu garip olabili...",anxiety,0.460065
